# Assignment

In this assignment we will train a convolutional neural network (CNN) on the CIFAR-10 dataset.

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook
* final trained `*.hdf5` model file
* final compiled `*.csv` file with performance statistics

# Google Colab

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [1]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

     |████████████████████████████████| 89 kB 3.7 MB/s 
     |████████████████████████████████| 596 kB 24.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Imports

Use the following lines to import any additional needed libraries:

In [2]:
import numpy as np, pandas as pd
from tensorflow.keras import Input, Model, models, layers, losses, metrics, optimizers
from jarvis.train import datasets

# Data

As in the tutorial, data for this assignment will consist of the CIFAR-10 dataset comprising 10 different everyday objects (airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck). The following lines of code will:

1. Download the dataset (if not already present) 
2. Prepare the necessary Python generators to iterate through dataset

In [3]:
# --- Download dataset
datasets.download(name='cifar')

# --- Prepare generators and model inputs
configs = {'batch': {'size': 36}}
gen_train, gen_valid, client = datasets.prepare(name='cifar', configs=configs, custom_layers=True)

[ 2022-04-14 22:53:45 ] [====================] 100.000% : Iterating | 000001    

*Note*: by default a batch size of `36` is used in this block of code. Feel free to experiment with larger or smaller values in assignment.

In [4]:
xs, _  = next(gen_train)

In [5]:
for k, v in xs.items():
    print('key = {} : shape = {}'.format(k.ljust(7), v.shape))

key = dat     : shape = (36, 32, 32, 3)
key = class   : shape = (36, 1)


# Training

In this assignment we will train a basic convolutional neural network on the CIFAR-10 dataset. At minumum you must include the following baseline techniques covered in the tutorial:

* convolutional operations
* batch normalization
* activation function
* subsampling

You are also **encouraged** to try different permuations and customizations to achieve optimal validation accuracy.

### Define backbone model

Feel free to use the `lambda` helper functions as demonstrated in the tutorial.

In [61]:
## --- Define input
x = Input(shape=(32,32,3), dtype='float32')

# --- Define model
kwargs = {
    'kernel_size': (3,3),
  'padding' : 'same'
}
conv = lambda x, filters, strides : layers.Conv2D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)

conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(2, 2))))

l1 = conv1(16, x)
l2 = conv1(24, conv2(24, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))

f0 = layers.Flatten()(l5)
h0 = 256
h1 = layers.Dense(h0, activation='relu')(f0)
h2 = layers.Dense(128, activation = 'relu')(h1)
h3 = layers.Dense(64, activation = 'relu')(h2)
h4 = layers.Dense(32, activation = 'relu')(h3)
# --- Define logits
logits = layers.Dense(10, name = 'class')(h4)

# --- Create model
backbone = Model(inputs=x, outputs=logits)

In [7]:
logits

<KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'class')>

In [62]:
scores = backbone.predict(xs['dat'])

### Define training model

In [63]:
# --- Define inputs
inputs = {
    'dat': Input(shape = (32, 32, 3), name = 'dat'),
    'class': Input(shape = (1,), name = 'class')}

# --- Define model
logits = backbone(inputs['dat'])

# --- Define loss
sce = losses.SparseCategoricalCrossentropy(from_logits=True)
print(sce(y_true=xs['class'], y_pred=scores))
loss = sce(y_true = inputs['class'], y_pred = logits)
# --- Define metric
acc = metrics.sparse_categorical_accuracy(y_true=inputs['class'], y_pred=logits)

tf.Tensor(2.302547, shape=(), dtype=float32)


Now, we are ready to create the `training` model and add the corresponding loss and accuracy tensors:

In [64]:
# --- Create model
training = Model(inputs=inputs, outputs={'logits': logits, 'loss': loss, 'acc': acc})

# --- Add loss
training.add_loss(loss)

# --- Add metric
training.add_metric(acc, name = 'acc')

### Compiling

Once the `training` model has been created, use the following to define an optimizer and compile:

In [65]:
# --- Define optimizer 
optimizer = optimizers.Adam(learning_rate=2e-4)

# --- Compile model
training.compile(optimizer = optimizer)

The model is now compiled and ready for training!

### Train the model

In [66]:
training.fit(
    x=gen_train, 
    steps_per_epoch=250, 
    epochs=50,
    validation_data=gen_valid,
    validation_steps=250,
    validation_freq=4
)

Epoch 1/50
250/250 [==============================] - 29s 108ms/step - loss: 2.1162 - acc: 0.1989
Epoch 2/50
250/250 [==============================] - 27s 109ms/step - loss: 1.8039 - acc: 0.3346
Epoch 3/50
250/250 [==============================] - 27s 110ms/step - loss: 1.6468 - acc: 0.3959
Epoch 4/50
250/250 [==============================] - 41s 162ms/step - loss: 1.5716 - acc: 0.4253 - val_loss: 1.5128 - val_acc: 0.4447
Epoch 5/50
250/250 [==============================] - 27s 110ms/step - loss: 1.4873 - acc: 0.4580
Epoch 6/50
250/250 [==============================] - 27s 109ms/step - loss: 1.4434 - acc: 0.4794
Epoch 7/50
250/250 [==============================] - 28s 111ms/step - loss: 1.4055 - acc: 0.4960
Epoch 8/50
250/250 [==============================] - 40s 160ms/step - loss: 1.3639 - acc: 0.5097 - val_loss: 1.4730 - val_acc: 0.4748
Epoch 9/50
250/250 [==============================] - 27s 110ms/step - loss: 1.3363 - acc: 0.5168
Epoch 10/50
250/250 [=======================

# Evaluation

Based on the tutorial discussion, use the following cells to check your algorithm performance. Consider loading a saved model and running prediction using `backbone.predict(...)` or `training.predict(...)` on the data aggregated via a test generator.

In [67]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True)

# --- Aggregate all examples
xs = {'dat' : [], 'class': []}

for x, _ in test_valid:
  xs['dat'].append(x['dat'])
  xs['class'].append(x['class'])

xs = {
    'dat': np.concatenate(xs['dat']),
    'class' : np.concatenate(xs['class'])
}
# --- Predict
outputs = training.predict(xs)
# --- Argmax
pred = np.argmax(outputs['logits'], axis = 1)

[ 2022-04-15 02:03:52 ] [====================] 100.000% : Iterating | 012000    

In [49]:
print(pred)

[9 1 7 ... 3 5 8]


In [68]:
df = pd.DataFrame(index=np.arange(pred.size))

# --- Define columns
df['true'] = xs['class']
df['pred'] = pred
df['corr'] = df['true'] == df['pred']

# --- Print accuracy
print(df['corr'].mean())

0.6298333333333334


**Note**: this cell is used only to check for model performance. It will not be graded. Once you are satisfied with your model, proceed to submission of your assignment below.

### Results

When ready, create a `*.csv` file with your compiled **validation** cohort statistics. There is no need to submit training performance accuracy. As in the tutorial, ensure that there are at least three columns in the `*.csv` file:

* true (ground-truth)
* pred (prediction)
* corr (correction prediction, True or False)

In [69]:
# --- Create *.csv

                              
# --- Serialize *.csv
df.to_csv('./wjhan_results.csv')

# Submission

Use the following line to save your model for submission:

In [70]:
# --- Serialize a model
backbone.save('./wjhan_model.hdf5')

### Canvas

Once you have completed this assignment, download the necessary files from Google Colab and your Google Drive. You will then need to submit the following items:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv`
* final (trained) model: `[UCInetID]_model.hdf5`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadshhet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.